In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import openpyxl
from datetime import datetime
import re

In [ ]:
# Get the number of last page avalaible
def get_last_news_page() -> int:
    url = "https://www.gamespot.com/news/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    btn = soup.find_all('a', class_='btn', recursive=True)
    pages = []
    for b in btn:
        try:
            p = int(b.text)
            pages.append(p)
        except:
            continue
    last_page = max(pages)
    return last_page

In [ ]:
# Get news between news_min and news_max. Returns dictionary with info about news (titles, links, dates)
def get_gamespot_news(last_page: int, news_min: int, news_max: int, display: bool = False) -> dict:    
    titles, links, dates, pages = [],[],[],[]
    url = "https://www.gamespot.com/news/"
    sum_news = 0
    for i in range(last_page):
        try:
            response = requests.get(url + "?page=" + str(last_page - i))
            soup = BeautifulSoup(response.text, 'html.parser')
            news = soup.find_all('div', class_="card-item__content")
            news = news[::-1]
            if (len(news) + sum_news) < news_min:
                sum_news += len(news)
                continue
            elif (len(news) + 120 + sum_news) < sum_news:
                sum_news += len(news) + 120
                i += 2
                continue
            for j in range(len(news)):
                if news_min <= sum_news + 1 <= news_max:
                    a = news[j].find('a', class_='card-item__link')
                    title = a.text
                    link = "https://www.gamespot.com" + str(a.get('href'))
                    date = news[j].find('time').get('datetime')
                    titles.append(title)
                    links.append(link)
                    dates.append(date)
                    pages.append(last_page - i)
                    if display == True:
                        if (sum_news - news_min + 1) % 100 == 0 and (sum_news - news_min + 1) != 0:
                            loaded = (sum_news - news_min + 1)
                            percent = round((100 * loaded)/(news_max - news_min + 1), 2)
                            print(loaded, "news loaded ( " + str(percent) + "% )")
                if sum_news > news_max:
                    final_dict = {}
                    final_dict['titles'] = titles
                    final_dict['links'] = links
                    final_dict['dates'] = dates
                    return final_dict
                
                sum_news += 1
        except:
            print("Error occured")
            break


In [ ]:
# Generate GameSpot news <from_value, to_value> where value 1 means first article ever. Multiplier is used to generate the same number of news in a sequence.
def generate_gamespot_news(from_value: int, to_value: int, multiplier: int = 1) -> None:
    folder_name = datetime.now()
    folder_name = folder_name.strftime('%Y_%m_%d %H-%M-%S')
    os.makedirs("Data/" + folder_name, exist_ok=True)
    last_page = get_last_news_page()
    for i in range(multiplier):
        l = from_value + (to_value * i)
        r = to_value + (to_value * i)
        dict_news = get_gamespot_news(last_page, l, r, True)
        df = pd.DataFrame(dict_news)
        df.to_csv("Data/" + folder_name + "/news" + str(l) + '-' + str(r) + ".csv", index=False)
        print("news", str(l) + '-' + str(r), "saved\n")

In [ ]:
# Create MS Excel file for given folder name which were generated before. Every portion of news (if multiplier = 1 then there is only one portion) has it's own sheet
def create_excel_file(target_folder_name: str) -> None:
    try:
        file_names = os.listdir("Data/" + target_folder_name)  
    except FileNotFoundError:
        print("No such directory")
        return

    os.makedirs("Data/Sheets", exist_ok=True)
    filenames_values = []
    csv_file_names = []

    for f in file_names:
        pattern = r'news(\d+)-(\d+)\.csv'
        match = re.match(pattern, f)
        if match:
            value1 = int(match.group(1))
            value2 = int(match.group(2))
            filenames_values.append(value1)
            filenames_values.append(value2)
            csv_file_names.append(f)

    if not csv_file_names:
        print("No matching CSV files found.")
        return

    excel_path = f"Data/Sheets/NewsSheet{target_folder_name} {min(filenames_values)}-{max(filenames_values)}.xlsx"
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for f in csv_file_names:
            df = pd.read_csv(f"Data/{target_folder_name}/{f}")
            df['rating'] = ''  # Add the 'rating' column without any default value (empty)
            sheet_name = f.replace(".csv", "")
            df.to_excel(writer, sheet_name=sheet_name, index=False)

            # Formatting with openpyxl
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            for row in worksheet.iter_rows():
                for cell in row:
                    cell.alignment = openpyxl.styles.Alignment(wrap_text=True)
                    # Convert URL text to hyperlinks if they look like URLs
                    if isinstance(cell.value, str) and (cell.value.startswith('http://') or cell.value.startswith('https://')):
                        cell.hyperlink = cell.value

            for col in worksheet.columns:
                max_length = 0
                column = col[0].column_letter
                for cell in col:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = (max_length + 2) * 1.2 
                worksheet.column_dimensions[column].width = adjusted_width

    print(f"Excel file created at {excel_path}")

# Example usage
create_excel_file("ExampleFolder")



In [ ]:
generate_gamespot_news(1, 1000, 3)

In [ ]:
target = '2024_05_26 20-09-45'
create_excel_file(target)